In [3]:
import os
from string import punctuation
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

In [4]:
df_temp = pd.read_excel('2_foidev_column_full_list_standardized.xlsx', sheetname='foidev_MANUFACTURER_full_list')
temp = list(df_temp['Standardized term'])
temp = list(set(temp))
temp.remove(np.nan)
temp.remove('UNK')
manufacturer = []
for i in temp:
    i = i.upper()
    i = i.replace(' CORP.', '').replace(' CORPRATION', '').replace(' LTD', '')\
        .replace(' INC.', '').replace(', INC.', '').replace(' INCORPORATED', '').strip(',')
    manufacturer.append(i)
    print(i)
del df_temp, temp

EUROMED
ALLEGIANCE HEALTHCARE
MCGHAN
SILIMED
MEDICAL ENGINEERING
INAMED
ALLERGAN
DOW CORNING
COX-UPHOFF INTERNATIONAL
IDEAL IMPLANT
NATURAL Y SURGICAL SPECIALTIES
NAGOR
POLY IMPLANT PROTHESE
MENTOR
BAXTER HEALTHCARE
SIENTRA
BIOPLASTY
HUTCHISON INTERNATIONAL
MEDICAL ENGINEERING CORPORATION
PMT
BIOSIL


In [5]:
fill_type = ['SALINE', 'SILICONE', 'GEL', 'COHESIVE']
surface_type = ['SMOOTH', 'TEXTURED', 'BIOCELL', 'MICROCELL', 'POLYURETHANE']
implantation_indication = ['AUGMENTATION', 'RECONSTRUCTION', 'COSMETIC', 'REVISION']
ALCL = ['ALCL', 'ANAPLASTIC LARGE CELL LYMPHOMA', 'LYMPHOMA', 'T-CELL LYMPHOMA', 'B-CELL LYMPHOMA',
        'CANCER', 'TUMOR', 'SUSPECT', 'CONFIRM']
side = ['LEFT', 'RIGHT', 'BILATERAL', 'BOTH SIDES']
biomarker = ['CD 30', 'ALK', 'NEGATIVE', 'POSITIVE', 'CD30-', 'CD30+', 'ALK-', 'ALK+']
symptom = ['breast pain', 'breast swelling', 'breast cyst', 'breast calcification', 'capsular contracture',
           'lymph node enlargement', 'firmness of breast', 'hematoma', 'mass', 'lump', 'rupture', 'deflated',
           'infection', 'abscess', 'leukopenia', 'nodules', 'skin discoloration', 'skin lesion', 'seroma',
           'effusion', 'fluid', 'edema', 'leak', 'redness', 'tenderness', 'erythema', 'asymmetry', 'breast enlargement']

In [15]:
punctuations = '''!()[]{};:'"\,<>./?@#$%^&*_~'''
stopwords = nltk.corpus.stopwords.words('english')

def processing_text(s):
    s = str(s)
    no_punct = ""
    for char in s:
        if char not in punctuations:
            no_punct = no_punct + char
    if ('ALCL-' in no_punct) | ('GEL-' in no_punct):
        no_punct = no_punct.replace('ALCL-', 'ALCL ')
        no_punct = no_punct.replace('GEL-', 'GEL ')
    l = no_punct.split(' ')
    output = []
    for w in l:
        if w in stopwords:
            continue
        output.append(w)
    return ' '.join(output)

In [16]:
df_BI_DEV = pd.read_csv('WF/DEV_BI_FULL_TABLE.txt', sep='|', header=0, encoding='ISO-8859-1', error_bad_lines=False)
df_BI_DEV = df_BI_DEV.drop_duplicates()
df_BI_TEXT = pd.read_csv('WF/TEXT_FULL_BI_LIST.txt', sep='|', header=0, encoding='ISO-8859-1', error_bad_lines=False)
df_BI_TEXT = df_BI_TEXT.drop_duplicates('MDR_TEXT_KEY')
print('Device #:', df_BI_DEV.shape[0])
print('Text #:', df_BI_TEXT.shape[0])
df_BI = df_BI_TEXT.merge(df_BI_DEV, on='MDR_REPORT_KEY', how='inner')
df_BI['MANUFACTURER_D_NAME'] = df_BI['MANUFACTURER_D_NAME'].astype(str)
df_BI['BRAND_NAME'] = df_BI['BRAND_NAME'].astype(str)
df_BI['GENERIC_NAME'] = df_BI['GENERIC_NAME'].astype(str)
df_BI['FOI_TEXT'] = df_BI['FOI_TEXT'].astype(str)
df_BI = df_BI.drop_duplicates()
df_BI['FOI_TEXT'] = df_BI['FOI_TEXT'].map(processing_text)
print('Full table #:', df_BI.shape[0])

Device #: 28182
Text #: 27137
Full table #: 28301


In [17]:
def text_search(text, k):
    l = text.split(' ')
    if ' ' in k:
        if k in text:
            return True
        else:
            return False
    else:
        if k in l:
            return True
        else:
            return False

def filter_merge(f1, f2, f3):
    return (f1 | f2 | f3)

In [18]:
print('Manufacturer word frequecy:\n')
for m in manufacturer:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['MANUFACTURER_D_NAME'], m)
    temp = df_BI.loc[df_BI['filter']==True, :]
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/manufacturer/'+m+'.txt', header=True, sep='|', index=False)
    print(m, ':', temp.shape[0])
    del temp

Manufacturer word frequecy:

EUROMED : 0
ALLEGIANCE HEALTHCARE : 14
MCGHAN : 2141
SILIMED : 7
MEDICAL ENGINEERING : 1542
INAMED : 199
ALLERGAN : 2565
DOW CORNING : 3630
COX-UPHOFF INTERNATIONAL : 2
IDEAL IMPLANT : 57
NATURAL Y SURGICAL SPECIALTIES : 1
NAGOR : 4
POLY IMPLANT PROTHESE : 3
MENTOR : 3502
BAXTER HEALTHCARE : 351
SIENTRA : 19
BIOPLASTY : 8
HUTCHISON INTERNATIONAL : 2
MEDICAL ENGINEERING CORPORATION : 3
PMT : 22
BIOSIL : 1681


In [19]:
print('Surface type word frequecy:\n')
for s in surface_type:
    df_BI['filter_1'] = np.vectorize(text_search)(df_BI['BRAND_NAME'], s)
    df_BI['filter_2'] = np.vectorize(text_search)(df_BI['GENERIC_NAME'], s)
    df_BI['filter_3'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], s)
    df_BI['filter'] = np.vectorize(filter_merge)(df_BI['filter_1'], df_BI['filter_2'], df_BI['filter_3'])
    temp = df_BI.loc[df_BI['filter']==True, :]
    del df_BI['filter'], df_BI['filter_1'], df_BI['filter_2'], df_BI['filter_3']
    temp.to_csv('WF_FULL_TABLE/surface/'+s+'.txt', header=True, sep='|', index=False)
    print(s, ':', temp.shape[0])
    del temp

Surface type word frequecy:

SMOOTH : 1032
TEXTURED : 769
BIOCELL : 75
MICROCELL : 1
POLYURETHANE : 121


In [20]:
print('Fill type word frequecy:\n')
for f in fill_type:
    df_BI['filter_1'] = np.vectorize(text_search)(df_BI['BRAND_NAME'], f)
    df_BI['filter_2'] = np.vectorize(text_search)(df_BI['GENERIC_NAME'], f)
    df_BI['filter_3'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], f)
    df_BI['filter'] = np.vectorize(filter_merge)(df_BI['filter_1'], df_BI['filter_2'], df_BI['filter_3'])
    temp = df_BI.loc[df_BI['filter']==True, :]
    del df_BI['filter'], df_BI['filter_1'], df_BI['filter_2'], df_BI['filter_3']
    temp.to_csv('WF_FULL_TABLE/fill/'+f+'.txt', header=True, sep='|', index=False)
    print(f, ':', temp.shape[0])
    del temp

Fill type word frequecy:

SALINE : 7378
SILICONE : 11189
GEL : 8281
COHESIVE : 363


In [21]:
print('Implantation indication word frequecy:\n')
for i in implantation_indication:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], i)
    temp = df_BI.loc[df_BI['filter']==True, :]
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/implantation_indication/'+i+'.txt', header=True, sep='|', index=False)
    print(i, ':', temp.shape[0])
    del temp

Implantation indication word frequecy:

AUGMENTATION : 1944
RECONSTRUCTION : 1053
COSMETIC : 257
REVISION : 316


In [22]:
print('ALCL word frequecy:\n')
for m in ALCL:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], m)
    temp = df_BI.loc[df_BI['filter']==True, :]
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/ALCL/'+m+'.txt', header=True, sep='|', index=False)
    print(m, ':', temp.shape[0])
    del temp

ALCL word frequecy:

ALCL : 721
ANAPLASTIC LARGE CELL LYMPHOMA : 492
LYMPHOMA : 905
T-CELL LYMPHOMA : 74
B-CELL LYMPHOMA : 1
CANCER : 828
TUMOR : 80
SUSPECT : 34
CONFIRM : 188


In [23]:
print('Side word frequecy:\n')
for s in side:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], s)
    temp = df_BI.loc[df_BI['filter']==True, :]
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/side/'+s+'.txt', header=True, sep='|', index=False)
    print(s, ':', temp.shape[0])
    del temp

Side word frequecy:

LEFT : 6917
RIGHT : 6975
BILATERAL : 6925
BOTH SIDES : 98


In [24]:
print('Biomarker word frequecy:\n')
for b in biomarker:
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], b)
    temp = df_BI.loc[df_BI['filter']==True, :]
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/biomarker/'+b+'.txt', header=True, sep='|', index=False)
    print(b, ':', temp.shape[0])
    del temp

Biomarker word frequecy:

CD 30 : 7
ALK : 89
NEGATIVE : 321
POSITIVE : 550
CD30- : 0
CD30+ : 79
ALK- : 65
ALK+ : 0


In [25]:
print('Symptom word frequecy:\n')
for s in symptom:
    s = s.upper()
    df_BI['filter'] = np.vectorize(text_search)(df_BI['FOI_TEXT'], s)
    temp = df_BI.loc[df_BI['filter']==True, :]
    del df_BI['filter']
    temp.to_csv('WF_FULL_TABLE/symptom/'+s+'.txt', header=True, sep='|', index=False)
    print(s, ':', temp.shape[0])
    del temp

Symptom word frequecy:

BREAST PAIN : 736
BREAST SWELLING : 35
BREAST CYST : 6
BREAST CALCIFICATION : 3
CAPSULAR CONTRACTURE : 3106
LYMPH NODE ENLARGEMENT : 3
FIRMNESS OF BREAST : 5
HEMATOMA : 348
MASS : 278
LUMP : 194
RUPTURE : 2846
DEFLATED : 1172
INFECTION : 1504
ABSCESS : 35
LEUKOPENIA : 2
NODULES : 78
SKIN DISCOLORATION : 13
SKIN LESION : 17
SEROMA : 782
EFFUSION : 43
FLUID : 668
EDEMA : 65
LEAK : 381
REDNESS : 149
TENDERNESS : 372
ERYTHEMA : 60
ASYMMETRY : 682
BREAST ENLARGEMENT : 10


In [26]:
full_list = {'manufacturer':manufacturer, 'fill':fill_type, 'surface':surface_type,
             'implantation_indication':implantation_indication, 'ALCL':ALCL, 'side':side,
             'biomarker':biomarker, 'symptom':symptom}

df_vector = pd.DataFrame()
df_vector['MDR_REPORT_KEY'] = df_BI['MDR_REPORT_KEY']

for key, value in full_list.items():
    for w in value:
        temp = pd.read_csv(os.path.join('WF_FULL_TABLE', key, w+'.txt'), sep='|', header=0,
                           encoding='ISO-8859-1', error_bad_lines=False)
        key_list = list(temp['MDR_REPORT_KEY'])
        del temp
        for k in key_list:
            df_vector.loc[df_vector['MDR_REPORT_KEY']==k, w] = 1
        try:
            df_vector[w] = df_vector[w].fillna(0)
        except:
            continue

for c in df_vector.columns.values:
    df_vector[c] = df_vector[c].astype(int)

In [27]:
print('Before:', df_vector.shape[0])
temp = df_BI[['MDR_REPORT_KEY', 'MANUFACTURER_D_NAME', 'BRAND_NAME', 'GENERIC_NAME', 'FOI_TEXT']]
df_vector = df_vector.merge(temp, on=['MDR_REPORT_KEY'], how='inner')
df_vector = df_vector.drop_duplicates()
print('After:', df_vector.shape[0])
df_vector.to_csv('keyword_vector.txt', sep='|', header=True, index=False)
df_vector.head(20)

Before: 28301
After: 27336


,MDR_REPORT_KEY,ALLEGIANCE HEALTHCARE,MCGHAN,SILIMED,MEDICAL ENGINEERING,INAMED,ALLERGAN,DOW CORNING,COX-UPHOFF INTERNATIONAL,IDEAL IMPLANT,...,leak,redness,tenderness,erythema,asymmetry,breast enlargement,MANUFACTURER_D_NAME,BRAND_NAME,GENERIC_NAME,FOI_TEXT
0,6730886,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,ALLERGAN (COSTA RICA),UNK SALINE IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",HEALTHCARE PROFESSIONAL REPORTED A LEFT SIDE D...
1,6730886,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,ALLERGAN (COSTA RICA),UNK SALINE IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",DEVICE EVALUATION VISUAL ANALYSIS OF THE RETUR...
4,6734192,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,ALLERGAN (COSTA RICA),STYLE 68 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",THE DEVICE REMAINS IMPLANTED A REVIEW OF THE D...
5,6734192,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,ALLERGAN (COSTA RICA),STYLE 68 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",PATIENT REPORTED THAT THE LEFT SIDE HAS COLLAP...
8,6283766,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,IDEAL IMPLANT,IDEAL IMPLANT STRUCTURED BREAST IMPLANT,SALINE-FILLED BREAST IMPLANT,DEFLATION RESULTING IN EXPLANTATION
9,6283766,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,IDEAL IMPLANT,IDEAL IMPLANT STRUCTURED BREAST IMPLANT,SALINE-FILLED BREAST IMPLANT,
12,6533466,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,MENTOR,MEMORY GEL BREAST IMPLANT,BREAST IMPLANT,I HAD A BREAST AUGMENTATION ON B6 2015 SINCE T...
16,6315557,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN (COSTA RICA),UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",THERAPEUTIC GOODS ADMINISTRATION REPORTED CONF...
17,6315557,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN (COSTA RICA),UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",THE EVENT OF LYMPHOMA IS A PHYSIOLOGICAL COMPL...
20,6747770,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,ALLERGAN (COSTA RICA),STYLE 68 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",A REVIEW OF THE DEVICE HISTORY RECORD HAS BEEN...


In [28]:
print('Is there saline and silicone at the same time?')
df_vector.loc[(df_vector['SALINE']==1) & (df_vector['SILICONE']==1), :]

Is there saline and silicone at the same time?


,MDR_REPORT_KEY,ALLEGIANCE HEALTHCARE,MCGHAN,SILIMED,MEDICAL ENGINEERING,INAMED,ALLERGAN,DOW CORNING,COX-UPHOFF INTERNATIONAL,IDEAL IMPLANT,...,leak,redness,tenderness,erythema,asymmetry,breast enlargement,MANUFACTURER_D_NAME,BRAND_NAME,GENERIC_NAME,FOI_TEXT
68,6553778,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,DUPONT/ DOW CORNING/ MENTOR,BREAST IMPLANT,BREAST IMPLANT,I HAD SILICONE IMPLANT IN 1986 AND LAST YEAR N...
69,6553778,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,DUPONT,BREAST IMPLANT,BREAST IMPLANT,I HAD SILICONE IMPLANT IN 1986 AND LAST YEAR N...
97,6751940,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN (COSTA RICA),UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",THIS EVENT IS A KNOWN POTENTIAL ADVERSE EVENT ...
98,6751940,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN (COSTA RICA),UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",HEALTHCARE PROFESSIONAL REPORTED UNKNOWN SIDE ...
125,6732140,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,ALLERGAN (COSTA RICA),UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",FURTHER INFORMATION FROM THE REPORTER REGARDIN...
126,6732140,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,ALLERGAN (COSTA RICA),UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",PATIENT REPORTED UNKNOWN SIDE RUPTURE DEVICE R...
181,6378780,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN (COSTA RICA),STYLE 363 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",IN RESPONSE TO FDA REPORT NUMBER MW5067421 THE...
182,6378780,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN (COSTA RICA),STYLE 363 SALINE FILLED BREAST IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",HEALTHCARE PROFESSIONAL REPORTED RIGHT SIDE SE...
201,6748708,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN (COSTA RICA),UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",THIS EVENT IS A KNOWN POTENTIAL ADVERSE EVENT ...
202,6748708,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN (COSTA RICA),UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",PATIENT REPORTED UNKNOWN SIDE LEAKING IMPLANT ...


In [29]:
print('Is there smooth and textured at the same time?')
df_vector.loc[(df_vector['SMOOTH']==1) & (df_vector['TEXTURED']==1), :]

Is there smooth and textured at the same time?


,MDR_REPORT_KEY,ALLEGIANCE HEALTHCARE,MCGHAN,SILIMED,MEDICAL ENGINEERING,INAMED,ALLERGAN,DOW CORNING,COX-UPHOFF INTERNATIONAL,IDEAL IMPLANT,...,leak,redness,tenderness,erythema,asymmetry,breast enlargement,MANUFACTURER_D_NAME,BRAND_NAME,GENERIC_NAME,FOI_TEXT
545,6498449,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,UNKNOWN MANUFACTURER,UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",FDA REPORTS Â¿THE FDA HAS RECEIVED A TOTAL OF ...
546,6498449,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,UNKNOWN MANUFACTURER,UNK MAMMARY IMPLANT,"PROSTHESIS, BREAST, INFLATABLE, INTERNAL, SALINE",THIS REPORT IS SUBMITTED IN RESPONSE TO FDA NO...
980,6567310,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,ALLERGAN (COSTA RICA),STYLE 410 COHESIVE SILICONE GEL FILLED BREAST ...,"PROSTHESIS, BREAST, NONINFLATABLE, INTERNAL, S...",ARTICLE CITATION Â¿CD30+ T CELLS IN LATE SEROM...
981,6567310,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,ALLERGAN (COSTA RICA),STYLE 410 COHESIVE SILICONE GEL FILLED BREAST ...,"PROSTHESIS, BREAST, NONINFLATABLE, INTERNAL, S...",REPORTED EVENT OF A PATIENT WITH A LEFT SIDE Â...
1148,6572184,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN,BREAST IMPLANT SILICONE STYLE 120 600CC,BREAST IMPLANT,RIGHT OPEN CAPSULECTOMY AND IMPLANT REMOVAL WI...
1149,6572184,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,ALLERGAN,"MENTOR SMOOTH ROUND PLUS STYLE, 1000 700CC",BREAST IMPLANT,RIGHT OPEN CAPSULECTOMY AND IMPLANT REMOVAL WI...
1278,6585642,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,MENTOR,SMOOTH SALINE BREAST IMPLANTS,SMOOTH SALINE BREAST IMPLANTS,MENTOR COMBO GEL SILICONESALINE MAMMARY PROSTH...
2760,6529881,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,MCGHAN/ALLERGAN,SALINE TEXTURED BREAST IMPLANT,BREAST IMPLANTS,I HAD A BREAST DEFORMITY AS A CHILDTEENAGER I ...
2762,6529881,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,MCGHAN/ALLERGAN,SILTEX SALINE FILLED MAMMARY PROSTHESIS,"PROSTHESIS, BREAST",I HAD A BREAST DEFORMITY AS A CHILDTEENAGER I ...
2852,6584830,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,MENTOR,GEL BREAST IMPLANTS,GEL BREAST IMPLANTS,APPROX 7 YEARS AGO I HAD A BREAST AUGMENTATION...
